### Restart and Run All

In [1]:
import calendar
import pandas as pd
import sidetable
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

# Define a formatting function to format integers with zero decimal places and floats with two decimal places
def format_cell(x):
    if isinstance(x, int):
        return "{:,.0f}".format(x)
    elif isinstance(x, float):
        return "{:.2f}".format(x)
    else:
        return x

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 6, 12)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,34,S,TFFIF,20000,7.7,1,DOS,SET,2023-05-31


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TFFIF,0.1032,0.1003,0.0920,0.1032,0.3987,"20,000",2023-05-31,2023-06-19,,1


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'TFFIF'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9739,TFFIF,8.50,7.00,S,7.65,7.70,24.39,0.41,7.65,"20,000",-4,4,"20,000",0,0,DOS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [7]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

35

In [8]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AWC,4500,4.74,0,52WL,SET50
1,B,BANPU,3000,7.80,0,RD30pct,SET50
2,O,BBL,200,125.00,0,,SET50
3,O,BDMS,1000,24.00,0,XXX,SET50
4,O,BEM,2500,7.80,0,52WL,SET50
5,O,BH,200,169.00,0,5pct,SET50
6,O,EA,400,61.00,0,52WL,SET50
7,O,FPT,1500,13.65,0,,SET
8,O,ICHI,3000,7.20,0,XXX,SET
9,B,IVL,800,30.00,0,RD20pct,SET50


In [9]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,S,PTTGC,600,53.25,0,RD45pct,SET50
1,S,JASIF,10000,8.50,0,CUT,SET
2,S,NER,5000,4.84,0,5pct,SET
3,I,BCH,3000,22.40,0,5pct,SET100
4,I,SENA,15000,4.50,0,C4.48,SET
5,I,GVREIT,10000,9.80,0,10pct,SET
6,I,CPNREIT,5000,13.20,0,5pct,SET
7,I,JMART,1200,20.60,0,5pct,SET50
8,S,TFFIF,20000,7.70,0,DOS,SET
9,I,AH,1200,39.00,0,5pct,SET


In [10]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(37, 7)

In [11]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,7,7,7,7,7,7
I,6,6,6,6,6,6
O,20,20,20,20,20,20
S,4,4,4,4,4,4


In [12]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,7,7,7,7,7,7
O,20,20,20,20,20,20
S,10,10,10,10,10,10


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(68, 2)

In [14]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(81, 9)

### If there is no dividend record, must create dividend record before proceed

In [15]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
0,O,AWC,4500.0,4.74,0.0,52WL,SET50,NaT,left_only
3,O,BDMS,1000.0,24.00,0.0,XXX,SET50,NaT,left_only
4,O,BEM,2500.0,7.80,0.0,52WL,SET50,NaT,left_only
5,O,BH,200.0,169.00,0.0,5pct,SET50,NaT,left_only
6,O,EA,400.0,61.00,0.0,52WL,SET50,NaT,left_only
7,O,FPT,1500.0,13.65,0.0,,SET,NaT,left_only
8,O,ICHI,3000.0,7.20,0.0,XXX,SET,NaT,left_only
13,O,KTC,400.0,50.00,0.0,52WL,SET50,NaT,left_only
15,O,LHFG,20000.0,1.00,0.0,,SET,NaT,left_only
17,O,OISHI,500.0,42.50,0.0,,SET,NaT,left_only


### Rerun from here

In [16]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,7,7,7,7,7,7,7
O,20,20,20,20,20,20,20
S,10,10,10,10,10,10,10


### Set xdate 2 weeks window of first Tuesday and second Friday

In [17]:
fm_date = '2023-03-28'

### Restart and Run All Cells

In [18]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
0,O,AWC,4500,4.74,0,52WL,SET50,NaT
2,O,BBL,200,125.00,0,,SET50,2023-04-21
3,O,BDMS,1000,24.00,0,XXX,SET50,NaT
4,O,BEM,2500,7.80,0,52WL,SET50,NaT
5,O,BH,200,169.00,0,5pct,SET50,NaT
6,O,EA,400,61.00,0,52WL,SET50,NaT
7,O,FPT,1500,13.65,0,,SET,NaT
8,O,ICHI,3000,7.20,0,XXX,SET,NaT
10,O,JMT,1800,39.75,0,52WL,SET50,2023-04-11
11,O,KKP,500,58.00,0,,SET100,2023-04-28


In [19]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
1,B,BANPU,3000,7.8,0,RD30pct,SET50,2023-04-10
9,B,IVL,800,30.0,0,RD20pct,SET50,2023-05-26
16,B,MCS,3000,7.0,0,RD50pct,SET,2023-03-17
21,B,SINGER,2000,10.4,0,RD60pct,SET100,2023-05-03
22,B,STA,1200,18.3,0,RD50pct,SET100,2023-04-19
23,B,SYNEX,3000,12.5,0,RND55pct,SET,2023-03-09
25,B,TMT,3000,7.6,0,RD25pct,SET,2023-04-18


In [20]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
36,S,AH,1200,39.00,0,5pct,SET,NaT
30,S,BCH,3000,22.40,0,5pct,SET100,2023-05-09
33,S,CPNREIT,5000,13.20,0,5pct,SET,2023-05-23
32,S,GVREIT,10000,9.80,0,10pct,SET,2023-05-26
28,S,JASIF,10000,8.50,0,CUT,SET,2023-05-19
34,S,JMART,1200,20.60,0,5pct,SET50,2023-04-11
29,S,NER,5000,4.84,0,5pct,SET,2023-04-20
27,S,PTTGC,600,53.25,0,RD45pct,SET50,2023-02-27
31,S,SENA,15000,4.50,0,C4.48,SET,2023-05-11
35,S,TFFIF,20000,7.70,0,DOS,SET,2023-05-31


In [21]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
1,B,BANPU,3000,7.80,0,RD30pct,SET50,2023-04-10
9,B,IVL,800,30.00,0,RD20pct,SET50,2023-05-26
16,B,MCS,3000,7.00,0,RD50pct,SET,2023-03-17
21,B,SINGER,2000,10.40,0,RD60pct,SET100,2023-05-03
22,B,STA,1200,18.30,0,RD50pct,SET100,2023-04-19
23,B,SYNEX,3000,12.50,0,RND55pct,SET,2023-03-09
25,B,TMT,3000,7.60,0,RD25pct,SET,2023-04-18
0,O,AWC,4500,4.74,0,52WL,SET50,NaT
2,O,BBL,200,125.00,0,,SET50,2023-04-21
3,O,BDMS,1000,24.00,0,XXX,SET50,NaT


In [22]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,27,27,27,27,27,27,27
S,10,10,10,10,10,10,10


In [23]:
rcds = df_orders.values.tolist()
len(rcds)

37

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'BANPU', 3000, 7.8, 0, 'RD30pct', 'SET50', '2023-04-10']
['B', 'IVL', 800, 30.0, 0, 'RD20pct', 'SET50', '2023-05-26']
['B', 'MCS', 3000, 7.0, 0, 'RD50pct', 'SET', '2023-03-17']
['B', 'SINGER', 2000, 10.4, 0, 'RD60pct', 'SET100', '2023-05-03']
['B', 'STA', 1200, 18.3, 0, 'RD50pct', 'SET100', '2023-04-19']
['B', 'SYNEX', 3000, 12.5, 0, 'RND55pct', 'SET', '2023-03-09']
['B', 'TMT', 3000, 7.6, 0, 'RD25pct', 'SET', '2023-04-18']
['B', 'AWC', 4500, 4.74, 0, '52WL', 'SET50', 'NaT']
['B', 'BBL', 200, 125.0, 0, '', 'SET50', '2023-04-21']
['B', 'BDMS', 1000, 24.0, 0, 'XXX', 'SET50', 'NaT']
['B', 'BEM', 2500, 7.8, 0, '52WL', 'SET50', 'NaT']
['B', 'BH', 200, 169.0, 0, '5pct', 'SET50', 'NaT']
['B', 'EA', 400, 61.0, 0, '52WL', 'SET50', 'NaT']
['B', 'FPT', 1500, 13.65, 0, '', 'SET', 'NaT']
['B', 'ICHI', 3000, 7.2, 0, 'XXX', 'SET', 'NaT']
['B', 'JMT', 1800, 39.75, 0, '52WL', 'SET50', '2023-04-11']
['B', 'KKP', 500, 58.0, 0, '', 'SET100', '2023-04-28']
['B', 'KTB', 1500, 14.3, 0, '', 'SET50', '20

In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [27]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BANPU,3000,7.80,0,RD30pct,SET50,2023-04-10
1,2,B,IVL,800,30.00,0,RD20pct,SET50,2023-05-26
2,3,B,MCS,3000,7.00,0,RD50pct,SET,2023-03-17
3,4,B,SINGER,2000,10.40,0,RD60pct,SET100,2023-05-03
4,5,B,STA,1200,18.30,0,RD50pct,SET100,2023-04-19
5,6,B,SYNEX,3000,12.50,0,RND55pct,SET,2023-03-09
6,7,B,TMT,3000,7.60,0,RD25pct,SET,2023-04-18
7,8,B,AWC,4500,4.74,0,52WL,SET50,NaT
8,9,B,BBL,200,125.00,0,,SET50,2023-04-21
9,10,B,BDMS,1000,24.00,0,XXX,SET50,NaT


In [28]:
df_orders.loc[(df_orders.xdate == 'NaT'),['xdate']] = ['2022-12-22']
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BANPU,3000,7.80,0,RD30pct,SET50,2023-04-10
1,2,B,IVL,800,30.00,0,RD20pct,SET50,2023-05-26
2,3,B,MCS,3000,7.00,0,RD50pct,SET,2023-03-17
3,4,B,SINGER,2000,10.40,0,RD60pct,SET100,2023-05-03
4,5,B,STA,1200,18.30,0,RD50pct,SET100,2023-04-19
5,6,B,SYNEX,3000,12.50,0,RND55pct,SET,2023-03-09
6,7,B,TMT,3000,7.60,0,RD25pct,SET,2023-04-18
7,8,B,AWC,4500,4.74,0,52WL,SET50,2022-12-22
8,9,B,BBL,200,125.00,0,,SET50,2023-04-21
9,10,B,BDMS,1000,24.00,0,XXX,SET50,2022-12-22


In [29]:
pd.read_sql_query('SELECT * FROM orders', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BANPU,3000,7.80,0,RD30pct,SET50,2023-04-10
1,2,B,IVL,800,30.00,0,RD20pct,SET50,2023-05-26
2,3,B,MCS,3000,7.00,0,RD50pct,SET,2023-03-17
3,4,B,SINGER,2000,10.40,0,RD60pct,SET100,2023-05-03
4,5,B,STA,1200,18.30,0,RD50pct,SET100,2023-04-19
5,6,B,SYNEX,3000,12.50,0,RND55pct,SET,2023-03-09
6,7,B,TMT,3000,7.60,0,RD25pct,SET,2023-04-18
7,8,B,AWC,4500,4.74,0,52WL,SET50,NaT
8,9,B,BBL,200,125.00,0,,SET50,2023-04-21
9,10,B,BDMS,1000,24.00,0,XXX,SET50,NaT


In [30]:
sqlUpd = """
UPDATE orders
SET xdate = '2022-12-22'
WHERE xdate = 'NaT'"""
rp = conlite.execute(sqlUpd)
rp.rowcount

13

In [31]:
pd.read_sql_query('SELECT * FROM orders WHERE qty = 0', conlite)

,id,trade,name,qty,price,active,reason,market,xdate


In [32]:
df_orders.shape

(37, 9)

In [33]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [34]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,8,B,AWC,4500,4.74,0,52WL,SET50,2022-12-22
1,1,B,BANPU,3000,7.80,0,RD30pct,SET50,2023-04-10
2,9,B,BBL,200,125.00,0,,SET50,2023-04-21
3,10,B,BDMS,1000,24.00,0,XXX,SET50,2022-12-22
4,11,B,BEM,2500,7.80,0,52WL,SET50,2022-12-22
5,12,B,BH,200,169.00,0,5pct,SET50,2022-12-22
6,13,B,EA,400,61.00,0,52WL,SET50,2022-12-22
7,2,B,IVL,800,30.00,0,RD20pct,SET50,2023-05-26
8,16,B,JMT,1800,39.75,0,52WL,SET50,2023-04-11
9,18,B,KTB,1500,14.30,0,,SET50,2023-04-20


In [35]:
name = 'BLA'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.000000,20.000000,20.000000,2.000000e+01,20.00000
mean,26.175000,26.525000,25.665000,3.145328e+06,26.18750
std,0.765627,0.760453,0.757437,1.902834e+06,0.68765
min,25.000000,25.250000,24.300000,6.837450e+05,25.00000
25%,25.437500,26.062500,25.000000,1.995296e+06,25.50000
50%,26.375000,26.625000,25.625000,2.697826e+06,26.25000
75%,27.000000,27.000000,26.312500,3.571551e+06,26.75000
max,27.250000,28.000000,26.750000,8.020689e+06,27.25000


### 2) Set100 records

In [36]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,17,B,KKP,500,58.0,0,,SET100,2023-04-28
1,23,B,ORI,3000,10.0,0,52WL,SET100,2023-05-08
2,24,B,PTG,1800,11.4,0,,SET100,2022-12-22
3,4,B,SINGER,2000,10.4,0,RD60pct,SET100,2023-05-03
4,5,B,STA,1200,18.3,0,RD50pct,SET100,2023-04-19
5,31,S,BCH,3000,22.4,0,5pct,SET100,2023-05-09


In [37]:
name = 'KKP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,63.287500,63.862500,62.745833,5.173294e+06,63.420833
std,1.475668,1.430913,1.480446,3.470447e+06,1.484019
min,59.250000,60.000000,58.750000,1.223683e+06,59.750000
25%,62.500000,63.250000,62.000000,2.724118e+06,62.750000
50%,63.125000,63.875000,62.625000,4.066109e+06,63.375000
75%,64.000000,64.500000,63.500000,6.102851e+06,64.250000
max,67.500000,67.750000,66.750000,1.695279e+07,67.000000


### 3) Set records

In [38]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,14,B,FPT,1500,13.65,0,,SET,2022-12-22
1,15,B,ICHI,3000,7.20,0,XXX,SET,2022-12-22
2,20,B,LANNA,1500,13.80,0,,SET,2023-03-18
3,21,B,LHFG,20000,1.00,0,,SET,2022-12-22
4,3,B,MCS,3000,7.00,0,RD50pct,SET,2023-03-17
5,22,B,OISHI,500,42.50,0,,SET,2022-12-22
6,6,B,SYNEX,3000,12.50,0,RND55pct,SET,2023-03-09
7,26,B,TASCO,1500,15.60,0,XXX,SET,2022-12-22
8,7,B,TMT,3000,7.60,0,RD25pct,SET,2023-04-18
9,37,S,AH,1200,39.00,0,5pct,SET,2022-12-22


In [39]:
name = 'SCCC'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
formatted_df = df.applymap(format_cell)

# Display the formatted dataframe
print(formatted_df)

     name        date   price    maxp    minp      qty    opnp
0    SCCC  2023-06-09  137.00  137.00  136.00   60,225  136.00
1    SCCC  2023-06-08  137.50  137.50  133.50  168,536  133.50
2    SCCC  2023-06-07  134.50  134.50  133.00   94,099  133.50
3    SCCC  2023-06-06  133.50  134.00  133.00   46,166  133.50
4    SCCC  2023-06-02  133.00  133.50  132.50   23,823  132.50
..    ...         ...     ...     ...     ...      ...     ...
115  SCCC  2022-12-15  149.50  149.50  148.50   35,194  149.00
116  SCCC  2022-12-14  149.00  149.50  148.50   19,329  149.00
117  SCCC  2022-12-13  149.00  149.50  148.00   74,035  148.50
118  SCCC  2022-12-09  148.50  149.00  148.00   77,233  148.50
119  SCCC  2022-12-08  148.50  148.50  148.00   26,034  148.50

[120 rows x 7 columns]
